In [1]:
import pandas as pd
from pme import pme_calc

## Load dataframes for CF, NAV and BENCH INDEX


In [6]:
cf_df = pd.read_csv('fund_cash_flows.csv')
cf_df['effective_date'] = pd.to_datetime(cf_df['effective_date'],format="%m/%d/%Y")

nav_df = pd.read_csv('fund_nav.csv')
nav_df['effective_date'] = pd.to_datetime(nav_df['effective_date'],format="%m/%d/%Y")

bench = pd.read_csv('bench_odce.csv')
bench['Period'] = pd.to_datetime(bench['Period'],format="%m/%d/%Y")



## Create a dataframe with datetime index that has columns for cash flows, nav and benchmark index

In [7]:
merge_df = pd.merge_ordered(
            nav_df[['pm_fund_id', 'effective_date', 'nav']],
            cf_df[['pm_fund_id', 'effective_date', 'cash_flow']],
            left_by="pm_fund_id"
        )
merge_df = (merge_df.loc[merge_df.effective_date <= '6/30/2020']
                .fillna(0)
                .sort_values('effective_date')
                .set_index('effective_date')
            )     


bench = bench[['Period', 'net_ret']].set_index('Period')
bench['bench'] = (1 + bench['net_ret']).cumprod()

#reindex bench to a daily index and interpolate so as to align the dates for cash flows
daily = pd.date_range(start = bench.index.min(), end = bench.index.max(), freq='D')
bench_daily = (bench[['bench']]
 .reindex(bench.index.union(daily))
 .interpolate(method='time')
 .reindex(daily)
)


merge_df = merge_df.join(bench_daily)


In [4]:
pme_calc(merge_df, '6/30/20')

{'irr': 0.09142615591881624,
 'tvpi': 1.320617960247663,
 'dpi': 0.2812078956013365,
 'dva': 83433438.35264146,
 'pme': 1.0871229530151707,
 'alpha': 0.02562145499793393,
 'bench_irr': 0.06381742798802459}